Let's start off by importing some packages we'll need.

In [ ]:
import numpy as np
import xarray as xr
from matplotlib import pyplot as plt
from tqdm import tqdm
import torch
from torch.utils.data import TensorDataset, DataLoader

* Create some random data of size (B, C, H) where B is a batch dimension, C is number of channels (start with 1) and H is height.
* Create a 1D convolution layer `L` using `torch.nn.Conv1d`, with a 3-element filter. Start off with 1 input and output.
* Inspect the free parameters of your layer using its `.weight` and `.bias` fields.
* How many dimensions does `L.weight` have, and what do they all represent?
* Set the bias and weights to all be zero. Set one weight to be 2. Apply your convolution layer to an input tensor and describe what happens.
* How many dimensions did your input tensor need to have, and what does each represent?

* Create and test a convolution with multiple input and output channels.

* Repeat the previous problem, but now for random input data of size (B, C, H, W) and a `torch.nn.Conv2d` layer with a 3x3 filter.
* Go up to at least 2 inputs and outputs. What does each dimension of `L.weight` and `L.bias` mean now?

Experiment with different modes for the `padding` argument to the Conv2d class. See the documentation [here](https://pytorch.org/docs/stable/generated/torch.nn.Conv2d.html).

* Create and test a `torch.nn.MaxPool2d` layer. Try several window sizes.
* Do the same for a `torch.nn.AvgPool2d` layer.

Try out the hyperbolic tangent function `torch.tanh`:
* Plot its output as a function of its input in increments of 0.01 from -10 to 10
* Plot the ReLU function on the same graph for comparison.
* Can you speculate as to when one or the other activation function might be preferable?

Download the sea surface temperature and upper ocean heat content fields for ENSO prediction. If you like, you can add these files to your google drive or local filesystem in the same place as your copy of this notebook, so that you won't have to install gdown and download them in the future whenever you restart this notebook.

In [ ]:
# ! executes shell commands instead of python commands

# library for downloading files from google drive.
!pip install gdown

In [ ]:
# enso index data from previous exercises
!gdown https://drive.google.com/uc?id=1FUb-2lcAd0Y1ULjx5jB6UTMNDmGy3vZA 
    
# sea surface temperature and upper ocean heat content
!gdown https://drive.google.com/uc?id=1TUFl4l4iEyIKTY1UnRD3G7fKElNMXjwo
!gdown https://drive.google.com/uc?id=1PkDoopBJdYdiVt_clxnbDfuDGJGM0QSY

Download the non-PCA data for ENSO prediction

Now load the sea surface temperature data (`sst.nc`) using xarray.

Convert the sea surface temperatures to a tensor with 32 bit floating point data. 

Load the target ENSO index data.

In [ ]:
# load the data
with np.load('enso_and_pca.npz') as data:    
    enso_index = data['y']  # 3-month-moving-averaged Nino3.4 index
    t_enso = data['t']  # months since jan. 1 1960 for the center of each 3-month window

The ENSO index is a 3-month moving average, and we'll try to predict it from temperature fields of of the ocean 3 months before the center of the window. That means we have to throw out
* The first few enso_index values, since we don't have ocean temperature fields 3 months earlier
* The last few temperature fields, since we don't have enso_index values 3 months later

Construct PyTorch tensors x and y for the temperature field in `sst` and the enso_index, so that `x[i]` was measured 3 months earlier than `y[i]`. Use as much of the data as you can, while dealing with the above-mentioned edge cases. To figure out the timing, compare `t_enso` and `sst.time`. Make sure `x` and `y` have a dtype of 32 bit floating point!

Normalize `x` and `y` to have a mean of zero and a standard deviation of 1. Don't normalize `x` separately for each location.

To use 2d convolutions, we need to have a channel dimension for our input tensor, which should be of size `(n_batch, n_channels, n_rows, n_columns)`. Use `reshape` or `unsqueeze` to add a channel axis of size 1 to `x`. It should be in position one, after the batch axis (position 0) but before the spatial dimensions.

Similarly, change `y` to have a second dimension of size 1, so that it has two dimensions total. This way it will match the output coming from out network.

Now create a TensorDataset and a dataloader from these tensors, just as we did in last week's exercises. Following Ham et al., we'll use a batch size of 400.

We won't split training and testing data this time so we can focus on what's new, but keep in mind this is normally the right practice.

Create a class defining your convolutional network.

Add code to the `__init__` method to define the layers, and add code to the `forward` method to use them.
* Start with a convolution with 30 output channels, and 'same' padding so that the image width and height are the same in the input and output. Use filters with a height of 4 and width of 8.
* Then use a `tanh` nonlinearity
* Next use a 2x2 maxpooling layer
* Then another convolution with 30 outputs, filter height 2 and filter width 4.
* Next another `tanh`
* Now another 2x2 maxpooling layer
* A third convolution with 30 outputs, filter height 2 and filter width 4
* Now reshape the output so the batch dimension is unchanged, but all other dimensions are combined into one.
* Now use a fully connected layer with 30 outputs
* Now a `tanh`
* Now a second fully connected layer with 1 output: that's $\hat our y$, which our forward method should return!

In [ ]:
# build the convnet from the paper
class ConvNet(torch.nn.Module):
    def __init__(self, input_channels=1, hidden_channels=30, hidden_units=30, n_outputs=1):
        super(ConvNet, self).__init__()

        #self.first_conv = 
        # ...

    def forward(self, x):
        #x = self.first_conv(x)
        #x = ...

        return x

Create an instance of your network class.

Pass a single batch of data through the network, and check that everything makes sense.

How many free parameters does your network have?

Write a training loop.

To avoid having to loop over trainable parameters, we'll use the optimizer class. We tell it once which parameters to update and what the learning rate is, then we can simply call `optimizer.step()` after calling `loss.backward()`, and all the parameters will be updated appropriately.

**Important**: For the optimizer to work properly each convolutional or fully connected layer must be an attribute of your `net.object`, for example by calling `self.layer = torch.nn.Conv2d(...)` in your network's `__init__` method. If you want to use a list instead that's ok, but in that case include `self.layers_list = torch.nn.ModuleList(layers_list)` to make sure that the optimizer finds the parameters of these layers.

Train your network over 100 epochs with a learning rate of 0.001. Keep track of the loss values and plot them at the end

**Optional**: if you want training to go more quickly, use a GPU instance. Generally you'll get it for 12 hours, and it will disconnect if it goes idle for an hour.
* In the colab menu, go to Runtime -> Change runtime type -> Hardware accelerator -> GPU and restart the colab notebook.
* Use `print(torch.cuda.get_device_name(0))` to figure out what kind of GPU you have
* Define `device = torch.device('cuda:0')`
* Call `net = net.to(device)` and `x = x.to(device)` and `y = y.to(device)`. Create a new Dataset and Dataloader, and start training


In [ ]:
n_epochs = 100
learning_rate = 0.001  # adjust this if necessary

lossfunc = torch.nn.MSELoss()
optimizer = torch.optim.SGD(net.parameters(), lr=learning_rate)

for epoch in tqdm(range(n_epochs)):
    for batch_index, batch in enumerate(data_loader):
        # note that each time this loop is run through, the order of the data is randomly permuted!
        x_batch, y_batch = batch
        optimizer.zero_grad()

        #yhat_batch = ...
        #loss = lossfunc( ... )
        
        loss.backward()
        optimizer.step()  # this will update parameters using torch.no_grad()
        optimzer.zero_grad()

Plot the predictions $\hat y$ against the targets $y$.

Calculate the Pearson's correlation between your prediction and the target over the training data.

Load `hc.nc`, containing heat content of the upper ocean, and use it as a second input channel to the network. Does this improve performance on the training data?

**Optional** Include two previous time steps as input for a total of 6 input channels.

In this case, we were able to load all of the training data into memory and store it in pytorch tensors. What if the training data were too big to fit into memory all at once? What could we do then? Propose a potential solution to this problem, and describe in words (or if you like, code) roughly how it would work.